# Модуль 9.3 — Сбор, хранение и визуализация

**Цель:** сохранять извлечённые сущности и связи, визуализировать их и готовить к интеграции с BI-инструментами.

**Что сделаем:**
- сохранение в JSON, SQLite, DataFrame
- визуализация графа связей (networkx)
- обзор интеграции с BI (Metabase, Superset, Power BI)

## 1. Сбор данных из NER/IE

Собираем сущности и связи в структурированный формат.

In [ ]:
import json
import pandas as pd

# Пример: результат NER
entities = [
    {"text": "Иван Петров", "label": "PER", "doc_id": "doc1"},
    {"text": "Яндекс", "label": "ORG", "doc_id": "doc1"},
    {"text": "Москва", "label": "LOC", "doc_id": "doc1"},
]

# Пример: результат IE
relations = [
    {"head": "Иван Петров", "type": "works_at", "tail": "Яндекс"},
    {"head": "Иван Петров", "type": "located_in", "tail": "Москва"},
]

print("Сущности:", entities)
print("Связи:", relations)

## 2. Сохранение: JSON и CSV

In [ ]:
# JSON
with open("entities_relations.json", "w", encoding="utf-8") as f:
    json.dump({"entities": entities, "relations": relations}, f, ensure_ascii=False, indent=2)

# DataFrame → CSV
df_ent = pd.DataFrame(entities)
df_rel = pd.DataFrame(relations)
df_ent.to_csv("entities.csv", index=False)
df_rel.to_csv("relations.csv", index=False)

print("Сохранили entities.csv и relations.csv")
display(df_ent)
display(df_rel)

## 3. SQLite для хранения

In [ ]:
import sqlite3

conn = sqlite3.connect("ie_store.db")
df_ent.to_sql("entities", conn, if_exists="replace", index=False)
df_rel.to_sql("relations", conn, if_exists="replace", index=False)
conn.close()

# Чтение
conn = sqlite3.connect("ie_store.db")
print(pd.read_sql("SELECT * FROM entities", conn))
conn.close()

## 4. Визуализация графа связей

In [ ]:
%pip install -q networkx matplotlib
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()
for e in entities:
    G.add_node(e["text"], label=e["label"])
for r in relations:
    G.add_edge(r["head"], r["tail"], type=r["type"])

plt.figure(figsize=(8, 5))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, font_size=10, node_color="lightblue",
        edge_color="gray", arrows=True, node_size=2000)
plt.title("Граф связей между сущностями")
plt.tight_layout()
plt.show()

## 5. Интеграция с BI-инструментами

**Metabase, Apache Superset, Power BI** подключаются к SQLite/PostgreSQL/CSV.

- Экспортируй данные в **PostgreSQL** или оставь в **SQLite** — Metabase и Superset читают оба.
- Power BI: загрузка через CSV или ODBC.
- Дашборд: таблицы сущностей, графики по типам, карта связей.

Пример подключения Metabase к SQLite: *Add database → SQLite → путь к ie_store.db*.

## Микро-упражнение

Построй граф связей из одного документа. Добавь 2–3 новые связи и перерисуй граф.